# Coding 2 assignment 3 

The following code shows the ssteps i took to scrape and clean data, which i used to to explore the effects of crude oil prices and market volatility on the S&P 500 Energy Sector Index. This project involved combining data from various sources, including Yahoo Finance, the Federal Reserve Economic Data (FRED) API, news sentiment data from EODHD API, and crude oil import data from the Energy Information Administration (EIA) API.


In [1]:
# %% importing modules 

from bs4 import BeautifulSoup
import requests
import pandas as pd
from scrapethat import *
import json
import requests
import ast  # To safely evaluate strings as Python objects
from tqdm import tqdm

In [2]:
#%% Processing s&P 500 energy sector data 

#setting parameters using curl
cookies = {
    'axids': 'gam=y-QsyyNKpE2uIyt3p_97gLJHSuT0u7nXie~A&dv360=eS1aZUZmQnhSRTJ1R3dCaGdROGE0dHM1NG1zNFVCcFg1M35B&ydsp=y-K63cpfpE2uJFw2RzeAH8ty3__rn0hX8h~A&tbla=y-_WANRaZE2uJkIoOlHotrTWo5vRNhyoPB~A',
    'tbla_id': 'b9a96955-51de-4df0-97ab-41b8db6f3600-tucte2e46da',
    'GUCS': 'ARs4LPVp',
    'GUC': 'AQABCAFnZy9nj0IcmgSZ&s=AQAAAH1WbZov&g=Z2Xrtw',
    'EuConsent': 'CQIBzgAQIBzgAAOACBENBUFoAP_gAEPgACiQKZNB9G7WTXFneXp2YPskOYUX0VBJ4MAwBgCBAcABzBIUIBwGVmAzJEyIICACGAIAIGBBIABtGAhAQEAAYIAFAABIAEgAIBAAIGAAACAAAABACAAAAAAAAAAQgEAXMBQgmAZEBFoIQUhAggAgAQAAAAAEAIgBCgQAEAAAQAAICAAIACgAAgAAAAAAAAAEAFAIEQAAAAECAotkfQAAAAAAAAAAAAAAAAABBTIAEg1KiAIsCQkIBAwggQAiCgIAKBAEAAAQIAAACYIChAGASowEQAgBAAAAAAAAAAQAIAAAIAEIAAgACBAAAAABAAEABAIAAAQAAAAAAAAAAAAAAAAAAAAAAAAAxACEEAAIAIIACCgAAAAEAAAAAAAAABEAAQAAAAAAAAAAAAABEAAAAAAAAAAAAAAAAAABAAAAAAAAAEAIgsAAAAAAAAAAAAAAAAAAIAA',
    'A1': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'A3': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'A1S': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'cmp': 't=1734732720&j=1&u=1---&v=57',
    'PRF': 't%3D%255EGSPE%252BNVDA%252BPCG%252BHLN%252BNCLH%252BAMCR%252BRIVN%252BAPI',
}

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    # 'cookie': 'axids=gam=y-QsyyNKpE2uIyt3p_97gLJHSuT0u7nXie~A&dv360=eS1aZUZmQnhSRTJ1R3dCaGdROGE0dHM1NG1zNFVCcFg1M35B&ydsp=y-K63cpfpE2uJFw2RzeAH8ty3__rn0hX8h~A&tbla=y-_WANRaZE2uJkIoOlHotrTWo5vRNhyoPB~A; tbla_id=b9a96955-51de-4df0-97ab-41b8db6f3600-tucte2e46da; GUCS=ARs4LPVp; GUC=AQABCAFnZy9nj0IcmgSZ&s=AQAAAH1WbZov&g=Z2Xrtw; EuConsent=CQIBzgAQIBzgAAOACBENBUFoAP_gAEPgACiQKZNB9G7WTXFneXp2YPskOYUX0VBJ4MAwBgCBAcABzBIUIBwGVmAzJEyIICACGAIAIGBBIABtGAhAQEAAYIAFAABIAEgAIBAAIGAAACAAAABACAAAAAAAAAAQgEAXMBQgmAZEBFoIQUhAggAgAQAAAAAEAIgBCgQAEAAAQAAICAAIACgAAgAAAAAAAAAEAFAIEQAAAAECAotkfQAAAAAAAAAAAAAAAAABBTIAEg1KiAIsCQkIBAwggQAiCgIAKBAEAAAQIAAACYIChAGASowEQAgBAAAAAAAAAAQAIAAAIAEIAAgACBAAAAABAAEABAIAAAQAAAAAAAAAAAAAAAAAAAAAAAAAxACEEAAIAIIACCgAAAAEAAAAAAAAABEAAQAAAAAAAAAAAAABEAAAAAAAAAAAAAAAAAABAAAAAAAAAEAIgsAAAAAAAAAAAAAAAAAAIAA; A1=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; A3=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; A1S=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; cmp=t=1734732720&j=1&u=1---&v=57; PRF=t%3D%255EGSPE%252BNVDA%252BPCG%252BHLN%252BNCLH%252BAMCR%252BRIVN%252BAPI',
    'origin': 'https://finance.yahoo.com',
    'priority': 'u=1, i',
    'referer': 'https://finance.yahoo.com/quote/%5EGSPE/history/?period1=736522200&period2=1734732330&frequency=1d&guccounter=1',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
}

params = {
    'events': 'capitalGain|div|split',
    'formatted': 'true',
    'includeAdjustedClose': 'true',
    'interval': '1d',
    'period1': '736522200',
    'period2': '1734732330',
    'symbol': '^GSPE',
    'userYfid': 'true',
    'lang': 'en-US',
    'region': 'US',
}
#running the get request
response = requests.get(
    'https://query1.finance.yahoo.com/v8/finance/chart/%5EGSPE',
    params=params,
    cookies=cookies,
    headers=headers,
)
t = response.text

# clean & save data to df 
# Decode and parse the response
if response.status_code == 200:
    # Decode raw byte string and parse JSON
    raw_data = response.content.decode('utf-8')
    json_data = json.loads(raw_data)
    
    # Extract the relevant part of the JSON
    chart_data = json_data.get('chart', {}).get('result', [])[0]
    timestamps = chart_data.get('timestamp', [])
    indicators = chart_data.get('indicators', {}).get('quote', [])[0]
    
    # Combine the data into a structured format
    df_energy = pd.DataFrame({
        'date': timestamps,
        'open': indicators.get('open', []),
        'close': indicators.get('close', []),
        'high': indicators.get('high', []),
        'low': indicators.get('low', []),
        'volume': indicators.get('volume', []),
    })
    
    # Convert the timestamp to a readable date format
    df_energy['date'] = pd.to_datetime(df_energy['date'], unit='s').dt.date
    
    # Sort the DataFrame by 'timestamp' in descending order
    df_energy = df_energy.sort_values(by='date', ascending=False)

    # Save to a CSV or print
    print(df_energy.head())
    #df.to_csv('finance_data.csv', index=False)
else:
    print(f"Request failed with status code {response.status_code}")

# aggregating from from daily to monthly frequency 
# did not direclty scrape mpnthly data cause it had missing values 
# Ensure 'date' is in datetime format
df_energy['date'] = pd.to_datetime(df_energy['date'])

# Set 'date' as the index for resampling
df_energy.set_index('date', inplace=True)

# Resample to monthly frequency and calculate aggregation
df_energy = df_energy.resample('MS').agg({
    'open': 'first',     # First available price of the month
    'close': 'last',     # Last available price of the month
    'high': 'max',       # Highest price during the month
    'low': 'min'         # Lowest price during the month
}).reset_index()

# Ensure the 'date' column is the first day of the month
df_energy['date'] = df_energy['date'].apply(lambda x: x.replace(day=1))

# Sort by date in descending order
df_energy = df_energy.sort_values(by='date', ascending=True)
df_energy = df_energy.reset_index()

# Display the result
df_energy.head()

            date        open       close        high         low     volume
7966  2024-12-20  632.559998  638.919983  640.580017  631.659973  390464000
7965  2024-12-19  645.789978  633.349976  646.840027  632.500000  193892700
7964  2024-12-18  654.580017  639.659973  658.260010  639.380005  170113600
7963  2024-12-17  655.440002  656.830017  658.119995  650.780029  178572600
7962  2024-12-16  675.119995  661.880005  675.119995  661.130005  158804600


,index,date,open,close,high,low
0,0,1993-05-01,101.830002,102.790001,103.070000,100.120003
1,1,1993-06-01,103.629997,101.699997,105.150002,100.900002
2,2,1993-07-01,101.379997,102.610001,102.760002,97.940002
3,3,1993-08-01,103.230003,105.900002,106.510002,100.309998
4,4,1993-09-01,105.930000,106.690002,107.059998,101.779999


In [3]:
# %% Processing VIX (market volitility) data  

#setting parameters 
cookies = {
    'axids': 'gam=y-QsyyNKpE2uIyt3p_97gLJHSuT0u7nXie~A&dv360=eS1aZUZmQnhSRTJ1R3dCaGdROGE0dHM1NG1zNFVCcFg1M35B&ydsp=y-K63cpfpE2uJFw2RzeAH8ty3__rn0hX8h~A&tbla=y-_WANRaZE2uJkIoOlHotrTWo5vRNhyoPB~A',
    'tbla_id': 'b9a96955-51de-4df0-97ab-41b8db6f3600-tucte2e46da',
    'GUC': 'AQABCAFnZy9nj0IcmgSZ&s=AQAAAH1WbZov&g=Z2Xrtw',
    'EuConsent': 'CQIBzgAQIBzgAAOACBENBUFoAP_gAEPgACiQKZNB9G7WTXFneXp2YPskOYUX0VBJ4MAwBgCBAcABzBIUIBwGVmAzJEyIICACGAIAIGBBIABtGAhAQEAAYIAFAABIAEgAIBAAIGAAACAAAABACAAAAAAAAAAQgEAXMBQgmAZEBFoIQUhAggAgAQAAAAAEAIgBCgQAEAAAQAAICAAIACgAAgAAAAAAAAAEAFAIEQAAAAECAotkfQAAAAAAAAAAAAAAAAABBTIAEg1KiAIsCQkIBAwggQAiCgIAKBAEAAAQIAAACYIChAGASowEQAgBAAAAAAAAAAQAIAAAIAEIAAgACBAAAAABAAEABAIAAAQAAAAAAAAAAAAAAAAAAAAAAAAAxACEEAAIAIIACCgAAAAEAAAAAAAAABEAAQAAAAAAAAAAAAABEAAAAAAAAAAAAAAAAAABAAAAAAAAAEAIgsAAAAAAAAAAAAAAAAAAIAA',
    'A1': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'A3': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'A1S': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'cmp': 't=1734781533&j=1&u=1---&v=57',
    'PRF': 't%3D%255EVIX%252B%255EGSPE%252BNVDA%252BPCG%252BHLN%252BNCLH%252BAMCR%252BRIVN%252BAPI',
}

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    # 'cookie': 'axids=gam=y-QsyyNKpE2uIyt3p_97gLJHSuT0u7nXie~A&dv360=eS1aZUZmQnhSRTJ1R3dCaGdROGE0dHM1NG1zNFVCcFg1M35B&ydsp=y-K63cpfpE2uJFw2RzeAH8ty3__rn0hX8h~A&tbla=y-_WANRaZE2uJkIoOlHotrTWo5vRNhyoPB~A; tbla_id=b9a96955-51de-4df0-97ab-41b8db6f3600-tucte2e46da; GUC=AQABCAFnZy9nj0IcmgSZ&s=AQAAAH1WbZov&g=Z2Xrtw; EuConsent=CQIBzgAQIBzgAAOACBENBUFoAP_gAEPgACiQKZNB9G7WTXFneXp2YPskOYUX0VBJ4MAwBgCBAcABzBIUIBwGVmAzJEyIICACGAIAIGBBIABtGAhAQEAAYIAFAABIAEgAIBAAIGAAACAAAABACAAAAAAAAAAQgEAXMBQgmAZEBFoIQUhAggAgAQAAAAAEAIgBCgQAEAAAQAAICAAIACgAAgAAAAAAAAAEAFAIEQAAAAECAotkfQAAAAAAAAAAAAAAAAABBTIAEg1KiAIsCQkIBAwggQAiCgIAKBAEAAAQIAAACYIChAGASowEQAgBAAAAAAAAAAQAIAAAIAEIAAgACBAAAAABAAEABAIAAAQAAAAAAAAAAAAAAAAAAAAAAAAAxACEEAAIAIIACCgAAAAEAAAAAAAAABEAAQAAAAAAAAAAAAABEAAAAAAAAAAAAAAAAAABAAAAAAAAAEAIgsAAAAAAAAAAAAAAAAAAIAA; A1=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; A3=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; A1S=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; cmp=t=1734781533&j=1&u=1---&v=57; PRF=t%3D%255EVIX%252B%255EGSPE%252BNVDA%252BPCG%252BHLN%252BNCLH%252BAMCR%252BRIVN%252BAPI',
    'origin': 'https://finance.yahoo.com',
    'priority': 'u=1, i',
    'referer': 'https://finance.yahoo.com/quote/%5EVIX/history/?period1=736387200&period2=1734735542&frequency=1d',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
}

params = {
    'events': 'capitalGain|div|split',
    'formatted': 'true',
    'includeAdjustedClose': 'true',
    'interval': '1d',
    'period1': '736387200',
    'period2': '1734735542',
    'symbol': '^VIX',
    'userYfid': 'true',
    'lang': 'en-US',
    'region': 'US',
}
#running the get request
response = requests.get(
    'https://query1.finance.yahoo.com/v8/finance/chart/%5EVIX',
    params=params,
    cookies=cookies,
    headers=headers,
)
#saving into a df 
if response.status_code == 200:
    # Decode raw byte string and parse JSON
    raw_data = response.content.decode('utf-8')
    json_data = json.loads(raw_data)
    
    # Extract the relevant part of the JSON
    chart_data = json_data.get('chart', {}).get('result', [])[0]
    timestamps = chart_data.get('timestamp', [])
    indicators = chart_data.get('indicators', {}).get('quote', [])[0]
    
    # Combine the data into a structured format
    df_VIX = pd.DataFrame({
        'date': timestamps,
        'open': indicators.get('open', []),
        'close': indicators.get('close', []),
        'high': indicators.get('high', []),
        'low': indicators.get('low', []),
        'volume': indicators.get('volume', []),
    })
    
    # Convert the timestamp to a readable date format
    df_VIX['date'] = pd.to_datetime(df_VIX['date'], unit='s').dt.date

    # Sort the DataFrame by 'timestamp' in descending order
    df_VIX = df_VIX.sort_values(by='date', ascending=False)

    print(df_VIX.head())
    #df.to_csv('finance_data.csv', index=False)
else:
    print(f"Request failed with status code {response.status_code}")

# aggregating from daily to monthly frequency 
# Ensure 'date' is in datetime format
df_VIX['date'] = pd.to_datetime(df_VIX['date'])

# Set 'date' as the index for resampling
df_VIX.set_index('date', inplace=True)

# Resample to monthly frequency and calculate Yahoo-like aggregation
df_VIX = df_VIX.resample('MS').agg({
    'open': 'first',     # First available price of the month
    'close': 'last',     # Last available price of the month
    'high': 'max',       # Highest price during the month
    'low': 'min'         # Lowest price during the month
}).reset_index()

# Ensure the 'date' column is the first day of the month
df_VIX['date'] = df_VIX['date'].apply(lambda x: x.replace(day=1))

# Sort by date in descending order
df_VIX = df_VIX.sort_values(by='date', ascending=True)
df_VIX = df_VIX.reset_index()
# Display the result
print(df_VIX.head())

            date       open      close       high    low  volume
8254  2024-12-20  24.139999  18.360001  26.510000  17.82     0.0
8253  2024-12-19  21.610001  24.090000  24.120001  20.16     0.0
8252  2024-12-18  15.570000  27.620001  28.320000  14.82     0.0
8251  2024-12-17  14.980000  15.870000  15.940000  14.78     0.0
8250  2024-12-16  14.370000  14.690000  14.690000  13.99     0.0
   index       date   open  close       high    low
0      0 1993-05-01  13.25  13.47  15.110000  11.96
1      1 1993-06-01  13.64  11.26  15.880000  10.86
2      2 1993-07-01  10.49  11.73  14.370000   9.11
3      3 1993-08-01  11.81  11.85  13.250000  10.80
4      4 1993-09-01  11.30  12.99  18.309999  10.75


In [4]:
# %% Testing the FRED (Federal reserve economic data)api   

# setting api key and series_id 
FRED_key = "301069c69bb232c0839c9faf237cd868"
series_id = 'DCOILWTICO'
url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={FRED_key}&file_type=json'
#running the get request
response = requests.get(url)
data = response.json()
# Access the observations
observations = data.get('observations', [])
for obs in observations:
    date = obs.get('date')
    value = obs.get('value')
    print(f'Date: {date}, Value: {value}')

Date: 1986-01-02, Value: 25.56
Date: 1986-01-03, Value: 26.00
Date: 1986-01-06, Value: 26.53
Date: 1986-01-07, Value: 25.85
Date: 1986-01-08, Value: 25.87
Date: 1986-01-09, Value: 26.03
Date: 1986-01-10, Value: 25.65
Date: 1986-01-13, Value: 25.08
Date: 1986-01-14, Value: 24.97
Date: 1986-01-15, Value: 25.18
Date: 1986-01-16, Value: 23.98
Date: 1986-01-17, Value: 23.63
Date: 1986-01-20, Value: 21.33
Date: 1986-01-21, Value: 20.61
Date: 1986-01-22, Value: 20.25
Date: 1986-01-23, Value: 19.93
Date: 1986-01-24, Value: 19.45
Date: 1986-01-27, Value: 20.87
Date: 1986-01-28, Value: 19.45
Date: 1986-01-29, Value: 19.61
Date: 1986-01-30, Value: 19.58
Date: 1986-01-31, Value: 18.95
Date: 1986-02-03, Value: 17.42
Date: 1986-02-04, Value: 15.58
Date: 1986-02-05, Value: 16.28
Date: 1986-02-06, Value: 16.60
Date: 1986-02-07, Value: 17.70
Date: 1986-02-10, Value: 16.78
Date: 1986-02-11, Value: 16.28
Date: 1986-02-12, Value: 15.74
Date: 1986-02-13, Value: 16.43
Date: 1986-02-14, Value: 16.03
Date: 19

In [5]:
# %% get crude oil price, interest rate, reer, consumption expenditure, cpi from FRED 

# FRED API key
FRED_key = "301069c69bb232c0839c9faf237cd868"

# Define the list of series to scrape
series_list = [
    {'id': 'MCOILWTICO', 'name': 'crude_oil_prices'},  # Crude oil prices
    {'id': 'FEDFUNDS', 'name': 'interest_rates'},      # Interest rates
    {'id': 'RBUSBIS', 'name': 'reer_exchange'},        # Reer Broad Effective Exchange Rate
    {'id': 'PCEPI', 'name': 'consumption_expenditure'}, # Personal Consumption Expenditures price index 
    {'id': 'CPIAUCSL', 'name': 'cpi'}                   #consumer price index - measure inflation 
]

# Function to fetch data from the FRED API
def fetch_fred_data(series_id, series_name):
    """
    Fetch data for a specific series from the FRED API and save it as a DataFrame.
    """
    try:
        # Construct the API URL
        url = f"https://api.stlouisfed.org/fred/series/observations"
        params = {
            'series_id': series_id,
            'api_key': FRED_key,
            'file_type': 'json'
        }
        
        # Make the API request
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise HTTPError for bad responses
        
        # Parse the JSON response
        data = response.json()
        observations = data.get('observations', [])
        
        # Convert observations to a DataFrame
        df = pd.DataFrame(observations)
        df['date'] = pd.to_datetime(df['date']).dt.date
        df.drop(columns=['realtime_start'], inplace=True)
        df.drop(columns=['realtime_end'], inplace=True)
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        df = df.rename(columns={'value': series['name']})
    
        # Save the DataFrame to a global variable
        globals()[series_name] = df
        print(f"Data for {series_name} fetched and saved successfully.")
    
    except Exception as e:
        print(f"Error fetching data for {series_name}: {e}")

# Loop through the series list and fetch data for each
for series in series_list:
    fetch_fred_data(series['id'], series['name'])

# Example: Display the DataFrames
print(crude_oil_prices.head())  
print(interest_rates.head())    
print(reer_exchange.head())
print(consumption_expenditure.head())
print(cpi.head())

#merge FRED DataFrames
merged_df = None
for series in series_list:
    # Access DataFrame dynamically by its name
    df = globals()[series['name']]
    # Merge on 'date'
    merged_df = df if merged_df is None else pd.merge(merged_df, df, on='date', how='outer')

# The final merged DataFrame is now stored in 'merged_df'
fred_df = merged_df
fred_df['date'] = pd.to_datetime(fred_df['date'])  # Ensure datetime type
print(fred_df['date'].dtypes)
fred_df.info()

Data for crude_oil_prices fetched and saved successfully.
Data for interest_rates fetched and saved successfully.
Data for reer_exchange fetched and saved successfully.
Data for consumption_expenditure fetched and saved successfully.
Data for cpi fetched and saved successfully.
         date  crude_oil_prices
0  1986-01-01             22.93
1  1986-02-01             15.46
2  1986-03-01             12.61
3  1986-04-01             12.84
4  1986-05-01             15.38
         date  interest_rates
0  1954-07-01            0.80
1  1954-08-01            1.22
2  1954-09-01            1.07
3  1954-10-01            0.85
4  1954-11-01            0.83
         date  reer_exchange
0  1994-01-01          88.69
1  1994-02-01          88.19
2  1994-03-01          88.30
3  1994-04-01          88.21
4  1994-05-01          87.45
         date  consumption_expenditure
0  1959-01-01                   15.164
1  1959-02-01                   15.179
2  1959-03-01                   15.189
3  1959-04-01      

In [6]:
#%% Getting company sentiments from EODHD api 

# dictonaires of comapnies in the sp500 enegry and their weights 
companies = [
    {"symbol": "XOM", "name": "ExxonMobil Corp", "weight": 21.84},
    {"symbol": "CVX", "name": "Chevron Corp", "weight": 15.46},
    {"symbol": "COP", "name": "ConocoPhillips", "weight": 7.99},
    {"symbol": "WMB", "name": "Williams Companies", "weight": 5.00},
    {"symbol": "EOG", "name": "EOG Resources", "weight": 4.46},
    {"symbol": "OKE", "name": "Oneok Inc", "weight": 4.46},
    {"symbol": "SLB", "name": "Schlumberger N.V.", "weight": 4.09},
    {"symbol": "KMI", "name": "Kinder Morgan", "weight": 3.96},
    {"symbol": "PSX", "name": "Phillips 66", "weight": 3.61},
    {"symbol": "MPC", "name": "Marathon Petroleum Corp", "weight": 3.44},
    {"symbol": "BKR", "name": "Baker Hughes Company", "weight": 3.09},
    {"symbol": "VLO", "name": "Valero Energy Corp", "weight": 2.98},
    {"symbol": "TRGP", "name": "Targa Resources", "weight": 2.95},
    {"symbol": "HES", "name": "Hess Corp", "weight": 2.76},
    {"symbol": "OXY", "name": "Occidental Petroleum Corp", "weight": 2.39},
    {"symbol": "FANG", "name": "Diamondback Energy", "weight": 2.24},
    {"symbol": "EQT", "name": "EQT Corp", "weight": 1.98},
    {"symbol": "HAL", "name": "Halliburton Company", "weight": 1.78},
    {"symbol": "DVN", "name": "Devon Energy Corp", "weight": 1.49},
    {"symbol": "CTRA", "name": "Coterra Energy Inc", "weight": 1.37},
    {"symbol": "TPL", "name": "Texas Pacific Land Trust", "weight": 1.33},
    {"symbol": "APA", "name": "APA Corp", "weight": 0.59},
]

# Function to fetch sentiment for a specific company
def get_company_sentiment(company_symbol):
    """
    Fetch sentiment data for a specific company using an API.
    """
    try:

        url = f'https://eodhd.com/api/sentiments?s={company_symbol}&from=1993-06-01&to=2024-12-01&api_token=6767148454d026.90284731&fmt=json'

        response = requests.get(url)
        
        if response.status_code == 200:
            return response.json()  # Return the raw JSON data
        else:
            print(f"Failed to fetch data for {company_symbol}, status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching sentiment for {company_symbol}: {e}")
        return None

# Function to process sentiment for companies and store in a list
def process_and_save_to_list(companies_list):
    """
    Fetch sentiment data for multiple companies and save to a list.
    """
    all_sentiments = []  # List to store all JSON responses

    for company in companies_list:
        company_symbol = company['symbol']
        print(f"Fetching data for {company_symbol}...")
        sentiment_data = get_company_sentiment(company_symbol)
        
        if sentiment_data:
            # Append the raw JSON data to the list
            all_sentiments.append({'symbol': company_symbol, 'data': sentiment_data})
            print(f"Data fetched for {company_symbol}")
        else:
            print(f"No data fetched for {company_symbol}.")
    
    return all_sentiments  # Return the list of all JSON responses

# Process the sentiment data
sentiments_list = process_and_save_to_list(companies)

# convert the list to a DataFrame 
sentiments_df = pd.DataFrame(sentiments_list)

# Inspect the resulting DataFrame or list
print(sentiments_df.head())  # Display the first few rows 
sentiments_df.to_csv('sentiment_data.csv', index=False)  # Save to a CSV file

Fetching data for XOM...
Data fetched for XOM
Fetching data for CVX...
Data fetched for CVX
Fetching data for COP...
Data fetched for COP
Fetching data for WMB...
Data fetched for WMB
Fetching data for EOG...
Data fetched for EOG
Fetching data for OKE...
Data fetched for OKE
Fetching data for SLB...
Data fetched for SLB
Fetching data for KMI...
Data fetched for KMI
Fetching data for PSX...
Data fetched for PSX
Fetching data for MPC...
Data fetched for MPC
Fetching data for BKR...
Data fetched for BKR
Fetching data for VLO...
Data fetched for VLO
Fetching data for TRGP...
Data fetched for TRGP
Fetching data for HES...
Data fetched for HES
Fetching data for OXY...
Data fetched for OXY
Fetching data for FANG...
Data fetched for FANG
Fetching data for EQT...
Data fetched for EQT
Fetching data for HAL...
Data fetched for HAL
Fetching data for DVN...
Data fetched for DVN
Fetching data for CTRA...
Data fetched for CTRA
Fetching data for TPL...
Data fetched for TPL
Fetching data for APA...
Dat

In [7]:
# %% adding company weights to the sentiments to use as a proxy for sentiment of our index 

sentiments_copy = sentiments_df.copy()
# Ensure your `data` column contains dictionaries and not strings
def parse_data_column(row):
    try:
        # If row is a string, evaluate it as a dictionary
        if isinstance(row, str):
            row = ast.literal_eval(row)  # Safely convert string to dict
        # Extract the first (and only) value from the dictionary
        return list(row.values())[0] if isinstance(row, dict) else None
    except Exception as e:
        print(f"Error parsing row: {row}, Error: {e}")
        return None

# Step 1: Parse the `data` column to ensure it contains lists of dictionaries
sentiments_copy['data'] = sentiments_copy['data'].apply(parse_data_column)

# Step 2: Explode the `data` column to create one row per dictionary
df_exploded = sentiments_copy.explode('data', ignore_index=True)

# Step 3: Normalize the exploded data into separate columns
df_sentiments_clean = pd.concat(
    [df_exploded.drop(columns=['data']),
     pd.json_normalize(df_exploded['data'])],
    axis=1
)
# Rename columns for clarity
df_sentiments_clean.rename(columns={'normalized': 'sentiment'}, inplace=True)
df_sentiments_clean['date'] = pd.to_datetime(df_sentiments_clean['date'])
# Display the resulting DataFrame
print(df_sentiments_clean)
df_sentiments_clean.describe()
#merge with weights 
weights_df = pd.DataFrame(companies)
df_sentiments_clean = df_sentiments_clean.merge(weights_df, on='symbol', how='left')
df_sentiments_clean
# Step 2: Extract the month from the date and encode as YYYY-MM-DD
df_sentiments_clean['month'] = pd.to_datetime(df_sentiments_clean['date']).dt.to_period('M')
# Step 3: Fill missing values with zeros for sentiment and count
df_sentiments_clean['sentiment'] = df_sentiments_clean['sentiment'].fillna(0)
df_sentiments_clean['count'] = df_sentiments_clean['count'].fillna(0)
# Step 4: Group by month and aggregate
sector_sentiment = (
    df_sentiments_clean.groupby('month')
    .apply(lambda x: pd.Series({
        'weighted_sentiment': (x['sentiment'] * x['weight']).sum() / x['weight'].sum(),
        'total_count': x['count'].sum()
    }))
    .reset_index()
)
# Rename 'month' to 'date' and ensure the format is YYYY-MM-DD with the first day of the month
sector_sentiment.rename(columns={'month': 'date'}, inplace=True)
sector_sentiment['date'] = sector_sentiment['date'].dt.to_timestamp()
# Display the updated DataFrame
print(sector_sentiment)
# classify the weighted sentiments into categories: positive, neutral and negative
def classify_sentiment(weighted_sentiment):
    if weighted_sentiment <= 0.4:
        return "Negative"
    elif weighted_sentiment <= 0.7:
        return "Neutral"
    else:
        return "Positive"
# Apply the function
sector_sentiment['sentiment_category'] = sector_sentiment['weighted_sentiment'].apply(classify_sentiment)
# Display the updated DataFrame
print(sector_sentiment)

      symbol       date  count  sentiment
0        XOM 2024-10-17      1     0.9990
1        XOM 2024-10-16      1     0.9990
2        XOM 2024-10-15      1     0.9970
3        XOM 2024-10-14      4     0.9985
4        XOM 2024-10-13      1     0.9980
...      ...        ...    ...        ...
14973    APA 2017-04-25      1     0.9980
14974    APA 2017-04-13      1     0.9370
14975    APA 2017-02-23      1     0.9490
14976    APA 2017-02-13      1     0.9020
14977    APA 2017-02-02      1     0.9370

[14978 rows x 4 columns]
          date  weighted_sentiment  total_count
0   2015-07-01            0.869312          7.0
1   2015-08-01            0.965651          7.0
2   2015-09-01            0.566746          8.0
3   2015-10-01            0.833438         10.0
4   2015-11-01            0.993904          7.0
..         ...                 ...          ...
108 2024-07-01            0.799530        658.0
109 2024-08-01            0.814914        510.0
110 2024-09-01            0.857591    

/var/folders/nx/lwphb_yx3312gqm2v4zgwnr00000gn/T/ipykernel_61697/8127799.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [ ]:
#%% Getting crude oil imports using us energy information adminsitration api  

all_data = [] 
def get_oil_imports(url):
    response = requests.get(url)
    data = response.json()
    observations = data['response']['data']
    for obs in observations:
        date = obs.get('period')
        value = obs.get('quantity')
        all_data.append({f'Date: {date}, quantity: {value}'})

# Generate URLs with offsets incrementing by 5000 up to 600,000
base_url = 'https://api.eia.gov/v2/crude-oil-imports/data/?frequency=monthly&data[0]=quantity&start=2009-01&end=2024-09&sort[0][column]=period&sort[0][direction]=desc&offset={}&length=600000&api_key=0vdyCAZCuAsoncBZ25DHYRvZ6x01Tx22AnVHsk7O'
urls = [base_url.format(offset) for offset in range(0, 600001, 5000)]
# Print the first few URLs as a sample
print(urls[:5])
#run get request - results saved to all_data list 
list(map(get_oil_imports, tqdm(urls)))
# Create a DataFrame
df_o = pd.DataFrame(all_data, columns=["Info"])
# Extract 'Date' and 'Quantity' using string manipulation
df_o["Date"] = df_o["Info"].str.extract(r"Date: ([\d-]+)")
df_o["Quantity"] = df_o["Info"].str.extract(r"quantity: (\d+)")
# Drop the original 'Info' column if not needed
df_o = df_o.drop(columns=["Info"])
# Display the resulting DataFrame
print(df_o)
# Ensure the date has a day component by appending '-01' if missing
df_o["Date"] = pd.to_datetime(df_o["Date"].astype(str) + "-01", format="%Y-%m-%d")
# Convert 'Quantity' to numeric
df_o["Quantity"] = pd.to_numeric(df_o["Quantity"])
#grouping by month and taking the sum 
df_o = df_o.groupby("Date", as_index=False).agg({"Quantity": "sum"})
# Display the result
print(df_o)
# visulising the data 
# Plot the data
plt.figure(figsize=(10, 6))
plt.plot(df_o["Date"], df_o["Quantity"], linestyle="-")
plt.title("Distribution of Quantity Over Time", fontsize=14)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Quantity", fontsize=12)
plt.grid(True)
plt.tight_layout()
# Show the plot
plt.show()

['https://api.eia.gov/v2/crude-oil-imports/data/?frequency=monthly&data[0]=quantity&start=2009-01&end=2024-09&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=600000&api_key=0vdyCAZCuAsoncBZ25DHYRvZ6x01Tx22AnVHsk7O', 'https://api.eia.gov/v2/crude-oil-imports/data/?frequency=monthly&data[0]=quantity&start=2009-01&end=2024-09&sort[0][column]=period&sort[0][direction]=desc&offset=5000&length=600000&api_key=0vdyCAZCuAsoncBZ25DHYRvZ6x01Tx22AnVHsk7O', 'https://api.eia.gov/v2/crude-oil-imports/data/?frequency=monthly&data[0]=quantity&start=2009-01&end=2024-09&sort[0][column]=period&sort[0][direction]=desc&offset=10000&length=600000&api_key=0vdyCAZCuAsoncBZ25DHYRvZ6x01Tx22AnVHsk7O', 'https://api.eia.gov/v2/crude-oil-imports/data/?frequency=monthly&data[0]=quantity&start=2009-01&end=2024-09&sort[0][column]=period&sort[0][direction]=desc&offset=15000&length=600000&api_key=0vdyCAZCuAsoncBZ25DHYRvZ6x01Tx22AnVHsk7O', 'https://api.eia.gov/v2/crude-oil-imports/data/?frequency=monthly&da

 61%|█████████████████████████▋                | 74/121 [07:09<05:56,  7.58s/it]

In [ ]:
# %% merging all dfs 

# Select the closing price from df_vix and df_energy, rename columns
df_VIX2 = df_VIX[['date', 'close']].rename(columns={'close': 'vix'})
df_energy2 = df_energy[['date', 'close']].rename(columns={'close': 's&p_500_energy'})
# Merge all data into a single DataFram
# Step 1: Rename `weighted_sentiment` to `sector_sentiment`
sector_sentiment.rename(columns={'weighted_sentiment': 'sector_sentiment'}, inplace=True)
# Step 2: Start the merge process
df = fred_df.copy()  # Start with with as the base DataFrame 
# Merge with thread_df
df = pd.merge(df, sector_sentiment, on='date', how='left')
# Merge with df_vix
df = pd.merge(df, df_VIX2, on='date', how='left')
# Merge with df_energy
df = pd.merge(df, df_energy2, on='date', how='left')
#merge with imports 
df = pd.merge(df, df_o, on='date', how='left')
# Step 3: Display the final DataFrame
print(df.head())

In [ ]:
# %% filtering the data 

df = df[df['date'] >= '1994-01-01']
# Reset the index after filtering 
df.reset_index(drop=True, inplace=True)
# Display the filtered DataFrame
print(df.head(15))

In [ ]:
# %% saving to csv 

df.to_csv('data.csv')

In [ ]:
#%% getting NYSE ARCA OIL and GAS INDEX (^XOI) for robustness check and external validity 

import requests

cookies = {
    'axids': 'gam=y-QsyyNKpE2uIyt3p_97gLJHSuT0u7nXie~A&dv360=eS1aZUZmQnhSRTJ1R3dCaGdROGE0dHM1NG1zNFVCcFg1M35B&ydsp=y-K63cpfpE2uJFw2RzeAH8ty3__rn0hX8h~A&tbla=y-_WANRaZE2uJkIoOlHotrTWo5vRNhyoPB~A',
    'tbla_id': 'b9a96955-51de-4df0-97ab-41b8db6f3600-tucte2e46da',
    'GUC': 'AQABCAFnZy9nj0IcmgSZ&s=AQAAAH1WbZov&g=Z2Xrtw',
    'A1': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'A3': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'cmp': 't=1735476616&j=1&u=1---&v=58',
    'EuConsent': 'CQIBzgAQIBzgAAOACBENBVFoAP_gAEPgACiQKZNB9G7WTXFneXp2YPskOYUX0VBJ4MAwBgCBAcABzBIUIBwGVmAzJEyIICACGAIAIGBBIABtGAhAQEAAYIAFAABIAEgAIBAAIGAAACAAAABACAAAAAAAAAAQgEAXMBQgmAZEBFoIQUhAggAgAQAAAAAEAIgBCgQAEAAAQAAICAAIACgAAgAAAAAAAAAEAFAIEQAAAAECAotkfQAAAAAAAAAAAAAAAAABBTIAEg1KiAIsCQkIBAwggQAiCgIAKBAEAAAQIAAACYIChAGASowEQAgBAAAAAAAAAAQAIAAAIAEIAAgACBAAAAABAAEABAIAAAQAAAAAAAAAAAAAAAAAAAAAAAAAxACEEAAIAIIACCgAAAAEAAAAAAAAABEAAQAAAAAAAAAAAAABEAAAAAAAAAAAAAAAAAABAAAAAAAAAEAIgsAAAAAAAAAAAAAAAAAAIAA',
    'A1S': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'PRF': 't%3D%255EXOI%252B%255EEST%252B%255EDWCREE%252B%255EVIX%252B%255EGSPE%252BNVDA%252BPCG%252BHLN%252BNCLH%252BAMCR%252BRIVN%252BAPI',
}

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    # 'cookie': 'axids=gam=y-QsyyNKpE2uIyt3p_97gLJHSuT0u7nXie~A&dv360=eS1aZUZmQnhSRTJ1R3dCaGdROGE0dHM1NG1zNFVCcFg1M35B&ydsp=y-K63cpfpE2uJFw2RzeAH8ty3__rn0hX8h~A&tbla=y-_WANRaZE2uJkIoOlHotrTWo5vRNhyoPB~A; tbla_id=b9a96955-51de-4df0-97ab-41b8db6f3600-tucte2e46da; GUC=AQABCAFnZy9nj0IcmgSZ&s=AQAAAH1WbZov&g=Z2Xrtw; A1=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; A3=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; cmp=t=1735476616&j=1&u=1---&v=58; EuConsent=CQIBzgAQIBzgAAOACBENBVFoAP_gAEPgACiQKZNB9G7WTXFneXp2YPskOYUX0VBJ4MAwBgCBAcABzBIUIBwGVmAzJEyIICACGAIAIGBBIABtGAhAQEAAYIAFAABIAEgAIBAAIGAAACAAAABACAAAAAAAAAAQgEAXMBQgmAZEBFoIQUhAggAgAQAAAAAEAIgBCgQAEAAAQAAICAAIACgAAgAAAAAAAAAEAFAIEQAAAAECAotkfQAAAAAAAAAAAAAAAAABBTIAEg1KiAIsCQkIBAwggQAiCgIAKBAEAAAQIAAACYIChAGASowEQAgBAAAAAAAAAAQAIAAAIAEIAAgACBAAAAABAAEABAIAAAQAAAAAAAAAAAAAAAAAAAAAAAAAxACEEAAIAIIACCgAAAAEAAAAAAAAABEAAQAAAAAAAAAAAAABEAAAAAAAAAAAAAAAAAABAAAAAAAAAEAIgsAAAAAAAAAAAAAAAAAAIAA; A1S=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; PRF=t%3D%255EXOI%252B%255EEST%252B%255EDWCREE%252B%255EVIX%252B%255EGSPE%252BNVDA%252BPCG%252BHLN%252BNCLH%252BAMCR%252BRIVN%252BAPI',
    'origin': 'https://finance.yahoo.com',
    'priority': 'u=1, i',
    'referer': 'https://finance.yahoo.com/quote/%5EXOI/history/?period1=757382400&period2=1733875200',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
}

params = {
    'events': 'capitalGain|div|split',
    'formatted': 'true',
    'includeAdjustedClose': 'true',
    'interval': '1d',
    'period1': '757382400',
    'period2': '1733875200',
    'symbol': '^XOI',
    'userYfid': 'true',
    'lang': 'en-US',
    'region': 'US',
}

response = requests.get(
    'https://query1.finance.yahoo.com/v8/finance/chart/%5EXOI',
    params=params,
    cookies=cookies,
    headers=headers,
)
t = response.text

# clean & save data to df 
# Decode and parse the response
if response.status_code == 200:
    # Decode raw byte string and parse JSON
    raw_data = response.content.decode('utf-8')
    json_data = json.loads(raw_data)
    
    # Extract the relevant part of the JSON
    chart_data = json_data.get('chart', {}).get('result', [])[0]
    timestamps = chart_data.get('timestamp', [])
    indicators = chart_data.get('indicators', {}).get('quote', [])[0]
    
    # Combine the data into a structured format
    df_xoi = pd.DataFrame({
        'date': timestamps,
        'open': indicators.get('open', []),
        'close': indicators.get('close', []),
        'high': indicators.get('high', []),
        'low': indicators.get('low', []),
        'volume': indicators.get('volume', []),
    })
    
    # Convert the timestamp to a readable date format
    df_xoi['date'] = pd.to_datetime(df_xoi['date'], unit='s').dt.date
    
    # Sort the DataFrame by 'timestamp' in descending order
    df_xoi = df_xoi.sort_values(by='date', ascending=False)

    # Save to a CSV or print
    print(df_xoi.head())
    #df.to_csv('finance_data.csv', index=False)
else:
    print(f"Request failed with status code {response.status_code}")

# aggregating from from daily to monthly frequency 
# did not direclty scrape mpnthly data cause it had missing values 
# Ensure 'date' is in datetime format
df_xoi['date'] = pd.to_datetime(df_xoi['date'])

# Set 'date' as the index for resampling
df_xoi.set_index('date', inplace=True)

# Resample to monthly frequency and calculate aggregation
df_xoi = df_xoi.resample('MS').agg({
    'open': 'first',     # First available price of the month
    'close': 'last',     # Last available price of the month
    'high': 'max',       # Highest price during the month
    'low': 'min'         # Lowest price during the month
}).reset_index()

# Ensure the 'date' column is the first day of the month
df_xoi['date'] = df_xoi['date'].apply(lambda x: x.replace(day=1))

# Sort by date in descending order
df_xoi = df_xoi.sort_values(by='date', ascending=True)
df_xoi = df_xoi.reset_index()

# Display the result
df_xoi.head()
df_xoi.shape
df_xoi.to_csv('XOI.csv')

In [ ]:
#%% Getting crude oil imports using us energy information adminsitration api  

all_data = [] 
def get_oil_imports(url):
    response = requests.get(url)
    data = response.json()
    observations = data['response']['data']
    for obs in observations:
        date = obs.get('period')
        value = obs.get('quantity')
        all_data.append({f'Date: {date}, quantity: {value}'})

# Generate URLs with offsets incrementing by 5000 up to 600,000
base_url = 'https://api.eia.gov/v2/crude-oil-imports/data/?frequency=monthly&data[0]=quantity&start=2009-01&end=2024-09&sort[0][column]=period&sort[0][direction]=desc&offset={}&length=600000&api_key=0vdyCAZCuAsoncBZ25DHYRvZ6x01Tx22AnVHsk7O'
urls = [base_url.format(offset) for offset in range(0, 600001, 5000)]
# Print the first few URLs as a sample
print(urls[:5])
#run get request - results saved to all_data list 
list(map(get_oil_imports, tqdm(urls)))
# Create a DataFrame
df_o = pd.DataFrame(all_data, columns=["Info"])
# Extract 'Date' and 'Quantity' using string manipulation
df_o["Date"] = df_o["Info"].str.extract(r"Date: ([\d-]+)")
df_o["Quantity"] = df_o["Info"].str.extract(r"quantity: (\d+)")
# Drop the original 'Info' column if not needed
df_o = df_o.drop(columns=["Info"])
# Display the resulting DataFrame
print(df_o)
# Ensure the date has a day component by appending '-01' if missing
df_o["Date"] = pd.to_datetime(df_o["Date"].astype(str) + "-01", format="%Y-%m-%d")
# Convert 'Quantity' to numeric
df_o["Quantity"] = pd.to_numeric(df_o["Quantity"])
#grouping by month and taking the sum 
df_o = df_o.groupby("Date", as_index=False).agg({"Quantity": "sum"})
# Display the result
print(df_o)


In [ ]:
# %% merging all dfs 
# Select the closing price from df_vix and df_energy, rename columns
df_VIX2 = df_VIX[['date', 'close']].rename(columns={'close': 'vix'})
df_energy2 = df_energy[['date', 'close']].rename(columns={'close': 's&p_500_energy'})
# Merge all data into a single DataFram
# Step 1: Rename `weighted_sentiment` to `sector_sentiment`
sector_sentiment.rename(columns={'weighted_sentiment': 'sector_sentiment'}, inplace=True)
# Step 2: Start the merge process
df = fred_df.copy()  # Start with with as the base DataFrame 
# Merge with thread_df
df = pd.merge(df, sector_sentiment, on='date', how='left')
# Merge with df_vix
df = pd.merge(df, df_VIX2, on='date', how='left')
# Merge with df_energy
df = pd.merge(df, df_energy2, on='date', how='left')
#merge with imports 
df_o.rename(columns={"Date": "date"}, inplace=True)
df = pd.merge(df, df_o, on='date', how='left')
# Step 3: Display the final DataFrame
print(df.head())

In [ ]:
# %% filtering the data 

df = df[df['date'] >= '1994-01-01']
# Reset the index after filtering 
df.reset_index(drop=True, inplace=True)
# Display the filtered DataFrame
print(df.head(15))

In [ ]:
# %% saving to csv 

df.to_csv('data.csv')

In [ ]:
#%% getting NYSE ARCA OIL and GAS INDEX (^XOI) for robustness check and external validity 

import requests

cookies = {
    'axids': 'gam=y-QsyyNKpE2uIyt3p_97gLJHSuT0u7nXie~A&dv360=eS1aZUZmQnhSRTJ1R3dCaGdROGE0dHM1NG1zNFVCcFg1M35B&ydsp=y-K63cpfpE2uJFw2RzeAH8ty3__rn0hX8h~A&tbla=y-_WANRaZE2uJkIoOlHotrTWo5vRNhyoPB~A',
    'tbla_id': 'b9a96955-51de-4df0-97ab-41b8db6f3600-tucte2e46da',
    'GUC': 'AQABCAFnZy9nj0IcmgSZ&s=AQAAAH1WbZov&g=Z2Xrtw',
    'A1': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'A3': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'cmp': 't=1735476616&j=1&u=1---&v=58',
    'EuConsent': 'CQIBzgAQIBzgAAOACBENBVFoAP_gAEPgACiQKZNB9G7WTXFneXp2YPskOYUX0VBJ4MAwBgCBAcABzBIUIBwGVmAzJEyIICACGAIAIGBBIABtGAhAQEAAYIAFAABIAEgAIBAAIGAAACAAAABACAAAAAAAAAAQgEAXMBQgmAZEBFoIQUhAggAgAQAAAAAEAIgBCgQAEAAAQAAICAAIACgAAgAAAAAAAAAEAFAIEQAAAAECAotkfQAAAAAAAAAAAAAAAAABBTIAEg1KiAIsCQkIBAwggQAiCgIAKBAEAAAQIAAACYIChAGASowEQAgBAAAAAAAAAAQAIAAAIAEIAAgACBAAAAABAAEABAIAAAQAAAAAAAAAAAAAAAAAAAAAAAAAxACEEAAIAIIACCgAAAAEAAAAAAAAABEAAQAAAAAAAAAAAAABEAAAAAAAAAAAAAAAAAABAAAAAAAAAEAIgsAAAAAAAAAAAAAAAAAAIAA',
    'A1S': 'd=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E',
    'PRF': 't%3D%255EXOI%252B%255EEST%252B%255EDWCREE%252B%255EVIX%252B%255EGSPE%252BNVDA%252BPCG%252BHLN%252BNCLH%252BAMCR%252BRIVN%252BAPI',
}

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    # 'cookie': 'axids=gam=y-QsyyNKpE2uIyt3p_97gLJHSuT0u7nXie~A&dv360=eS1aZUZmQnhSRTJ1R3dCaGdROGE0dHM1NG1zNFVCcFg1M35B&ydsp=y-K63cpfpE2uJFw2RzeAH8ty3__rn0hX8h~A&tbla=y-_WANRaZE2uJkIoOlHotrTWo5vRNhyoPB~A; tbla_id=b9a96955-51de-4df0-97ab-41b8db6f3600-tucte2e46da; GUC=AQABCAFnZy9nj0IcmgSZ&s=AQAAAH1WbZov&g=Z2Xrtw; A1=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; A3=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; cmp=t=1735476616&j=1&u=1---&v=58; EuConsent=CQIBzgAQIBzgAAOACBENBVFoAP_gAEPgACiQKZNB9G7WTXFneXp2YPskOYUX0VBJ4MAwBgCBAcABzBIUIBwGVmAzJEyIICACGAIAIGBBIABtGAhAQEAAYIAFAABIAEgAIBAAIGAAACAAAABACAAAAAAAAAAQgEAXMBQgmAZEBFoIQUhAggAgAQAAAAAEAIgBCgQAEAAAQAAICAAIACgAAgAAAAAAAAAEAFAIEQAAAAECAotkfQAAAAAAAAAAAAAAAAABBTIAEg1KiAIsCQkIBAwggQAiCgIAKBAEAAAQIAAACYIChAGASowEQAgBAAAAAAAAAAQAIAAAIAEIAAgACBAAAAABAAEABAIAAAQAAAAAAAAAAAAAAAAAAAAAAAAAxACEEAAIAIIACCgAAAAEAAAAAAAAABEAAQAAAAAAAAAAAAABEAAAAAAAAAAAAAAAAAABAAAAAAAAAEAIgsAAAAAAAAAAAAAAAAAAIAA; A1S=d=AQABBJnJNGcCEIlOtB7AFhfcTs7EADdShusFEgABCAEvZ2ePZ-7ubmUB9qMAAAcIfTkqZ8fohiE&S=AQAAAghyKKKrUBAHUUsY0CjDd_E; PRF=t%3D%255EXOI%252B%255EEST%252B%255EDWCREE%252B%255EVIX%252B%255EGSPE%252BNVDA%252BPCG%252BHLN%252BNCLH%252BAMCR%252BRIVN%252BAPI',
    'origin': 'https://finance.yahoo.com',
    'priority': 'u=1, i',
    'referer': 'https://finance.yahoo.com/quote/%5EXOI/history/?period1=757382400&period2=1733875200',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
}

params = {
    'events': 'capitalGain|div|split',
    'formatted': 'true',
    'includeAdjustedClose': 'true',
    'interval': '1d',
    'period1': '757382400',
    'period2': '1733875200',
    'symbol': '^XOI',
    'userYfid': 'true',
    'lang': 'en-US',
    'region': 'US',
}

response = requests.get(
    'https://query1.finance.yahoo.com/v8/finance/chart/%5EXOI',
    params=params,
    cookies=cookies,
    headers=headers,
)
t = response.text

# clean & save data to df 
# Decode and parse the response
if response.status_code == 200:
    # Decode raw byte string and parse JSON
    raw_data = response.content.decode('utf-8')
    json_data = json.loads(raw_data)
    
    # Extract the relevant part of the JSON
    chart_data = json_data.get('chart', {}).get('result', [])[0]
    timestamps = chart_data.get('timestamp', [])
    indicators = chart_data.get('indicators', {}).get('quote', [])[0]
    
    # Combine the data into a structured format
    df_xoi = pd.DataFrame({
        'date': timestamps,
        'open': indicators.get('open', []),
        'close': indicators.get('close', []),
        'high': indicators.get('high', []),
        'low': indicators.get('low', []),
        'volume': indicators.get('volume', []),
    })
    
    # Convert the timestamp to a readable date format
    df_xoi['date'] = pd.to_datetime(df_xoi['date'], unit='s').dt.date
    
    # Sort the DataFrame by 'timestamp' in descending order
    df_xoi = df_xoi.sort_values(by='date', ascending=False)

    # Save to a CSV or print
    print(df_xoi.head())
    #df.to_csv('finance_data.csv', index=False)
else:
    print(f"Request failed with status code {response.status_code}")

# aggregating from from daily to monthly frequency 
# did not direclty scrape mpnthly data cause it had missing values 
# Ensure 'date' is in datetime format
df_xoi['date'] = pd.to_datetime(df_xoi['date'])

# Set 'date' as the index for resampling
df_xoi.set_index('date', inplace=True)

# Resample to monthly frequency and calculate aggregation
df_xoi = df_xoi.resample('MS').agg({
    'open': 'first',     # First available price of the month
    'close': 'last',     # Last available price of the month
    'high': 'max',       # Highest price during the month
    'low': 'min'         # Lowest price during the month
}).reset_index()

# Ensure the 'date' column is the first day of the month
df_xoi['date'] = df_xoi['date'].apply(lambda x: x.replace(day=1))

# Sort by date in descending order
df_xoi = df_xoi.sort_values(by='date', ascending=True)
df_xoi = df_xoi.reset_index()

# Display the result
df_xoi.head()
df_xoi.shape
df_xoi.to_csv('XOI.csv')